In [1]:
import numpy as np

In [2]:
np.set_printoptions(precision=4, suppress=True)

In [ ]:
# Asymmetric range
params = np.random.uniform(low=-50, high=150, size=10)

params[0] = 0
params[1] = np.min(params) - 1 # min value
params[2] = np.max(params) + 1 # max value

params = np.round(params, decimals=4)

params

array([  0.    ,  -8.5935, 133.9665,  32.5414, 132.9665,  57.6461,
        55.0637,  -3.2304,  30.2418,  -7.5935])

In [20]:
# ===== Symmetric Quantization =====

def symmetric_quantization(params, bits):
    alpha = np.max(np.abs(params))
    # strictly symmetrical bounds
    lower_limit = - (2**(bits - 1) - 1)
    upper_limit = 2**(bits - 1) - 1 
    # scale
    scale = alpha / upper_limit

    q_params = np.round(params / scale).astype(int)
    q_params = np.clip(q_params, lower_limit, upper_limit)

    return q_params, scale

def symmetric_dequantization(q_params, scale):
    return q_params * scale

# ===== Assymetric Quantization ====

def asymmetric_quantization(params, bits):
    alpha = np.max(params)
    beta = np.min(params)
    # bounds
    lower_limit = 0
    upper_limit = 2**bits - 1
    # scale
    scale = (alpha - beta) / upper_limit
    # zero
    zero = np.round(-1 * np.round(beta / scale))

    q_params = np.round(params / scale + zero).astype(int)
    q_params = np.clip(q_params, lower_limit, upper_limit)

    return q_params, scale, zero

def asymmetric_dequantization(q_params, scale, zero):
    return (q_params - zero) * scale

# calculate quantization error
def mse(params, q_params):
    return (params - q_params) ** 2

In [21]:
q_params, scale = symmetric_quantization(params, bits=8)
dq_params = symmetric_dequantization(q_params, scale)
q_error = mse(params, dq_params)

print(f"Parameters:")
print(params)
print("")
print(f"Quantized Symmetric:")
print(dq_params)
print(f"scale: {scale}")
print("")
print(f"Dequantized Symmetric: {dq_params}")
print("")
print(f"Quantized error: {np.mean(q_error):.4f}")


Parameters:
[  0.      -8.5935 133.9665  32.5414 132.9665  57.6461  55.0637  -3.2304
  30.2418  -7.5935]

Quantized Symmetric:
[  0.      -8.4388 133.9665  32.7005 132.9116  58.017   54.8524  -3.1646
  30.5908  -7.384 ]
scale: 1.0548543307086613

Dequantized Symmetric: [  0.      -8.4388 133.9665  32.7005 132.9116  58.017   54.8524  -3.1646
  30.5908  -7.384 ]

Quantized error: 0.0404


In [22]:
q_params, scale, zero = asymmetric_quantization(params, bits=8)
dq_params = asymmetric_dequantization(q_params, scale, zero)
q_error = mse(params, dq_params)


print(f"Parameters:")
print(params)
print("")
print(f"Quantized Asymmetric:")
print(dq_params)
print(f"scale: {scale}")
print("")
print(f"Dequantized Asymmetric: {dq_params}")
print("")
print(f"Quantized error: {np.mean(q_error):.4f}")


Parameters:
[  0.      -8.5935 133.9665  32.5414 132.9665  57.6461  55.0637  -3.2304
  30.2418  -7.5935]

Quantized Asymmetric:
[  0.      -8.3859 134.1741  32.4254 133.056   57.5831  54.7878  -3.3544
  30.1892  -7.8268]
scale: 0.5590588235294117

Dequantized Asymmetric: [  0.      -8.3859 134.1741  32.4254 133.056   57.5831  54.7878  -3.3544
  30.1892  -7.8268]

Quantized error: 0.0260
